In [1]:

import os
os.chdir("..")

import asyncio
from dotenv import load_dotenv
import shutil
import subprocess
import requests
import time
import os

from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
from langchain_openai import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.chains import LLMChain
 



prompt_template = """Given the following information:

1. User details Json: {user_details_json}


Task:
* If user_details_json is not empty
    * Extract name and relevant past history.
    * Craft a personalized response (max 2 lines) based on the current chat and history.
    * Prioritize using the name and a warm greeting.
* IF JSON DOES NOT EXIST:
    * Craft a general, friendly response (max 2 lines) to continue the conversation.

Example JSON (if exists):
{
  "name": "Alice",
  "past_history": ["Asked about product features", "Expressed interest in discounts"]
}"""


from langchain.prompts import FewShotPromptTemplate
from langchain.prompts import PromptTemplate



prompt = PromptTemplate(
    template=prompt_template,
    input_variables=['user_details_json']
)

text = """{
  "name": "Alice",
  "past_history": ["Asked about product features", "Expressed interest in discounts"]
}"""



llm = ChatGroq(temperature=0, model_name="mixtral-8x7b-32768", groq_api_key=os.getenv("GROQ_API_KEY"))
 



from langchain_community.chat_message_histories import SQLChatMessageHistory





In [2]:
def get_session_history(session_id):
    return SQLChatMessageHistory(session_id, "sqlite:///memory.db")

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory


prompt_template = """You are a conversational medical assistant.
Use short, conversational responses as if you're having a live conversation. Restrict
yourself from using unnecessary responses. Keep the responses short and precise.

Given the following information:
User details Json: {user_details_json}


Task:
* Extract name and relevant past history.
* Craft a personalized response (max 2 lines) based on the current chat and history.
* Prioritize using the name and a warm greeting.


If user_details_json is None, then the history will be returned as a JSON string. Otherwise it will be returned as a JSON string with the 
following fields filled in: name and description of the current chat and history. 
"""

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            prompt_template
        ),
        
        MessagesPlaceholder(variable_name="history"),
        ("human", "{input}"),
    ]
)

runnable = prompt | llm



In [3]:
runnable_with_history = RunnableWithMessageHistory(
    runnable,
    get_session_history,
    
    input_messages_key="input",
    history_messages_key="history",
)



In [4]:

runnable_with_history.invoke(
    {"user_details_json":
        """{"name": "Rishi",
        "past_history": ["Asked about product features", "Expressed interest in discounts"]}""",
        
        
     "input": "HI!"},
    config={"configurable": {"session_id": "2"}},
)


AIMessage(content='Hello there! I\'m glad you\'re here, Rishi. I\'m here to help you with any questions or concerns you might have. Is there anything specific you would like to know about our medical platform or any other topic? 😊 \n\n(Note: I assumed you might have meant "Hi" as a greeting, so I responded accordingly. If you meant something else, please let me know!)')

In [5]:

runnable_with_history.invoke(
    {"user_details_json":
        None,
        
     "input": "My name is rishi and i am having fever!"},
    config={"configurable": {"session_id": "1"}},
)



AIMessage(content="Hello Rishi, great to e-meet you! I see you're dealing with a fever, I'm here to help. Let's discuss your symptoms and how I can assist you further.")

In [6]:
runnable_with_history.invoke(
    {"user_details_json":
        None,
        
     "input": "Can you tell me what questions did i ask"},
    config={"configurable": {"session_id": "2"}},
)



AIMessage(content="I apologize for any confusion, Rishi. Since I don't have access to our previous conversations, I can't provide you with the specific questions you asked before. However, I'm here to help you with any questions or concerns you might have now. Is there anything specific you would like to know about our medical platform or any other topic? 😊")

In [7]:
runnable_with_history.invoke(
    {"user_details_json":
        None,
        
     "input": "Hi"},
    config={"configurable": {"session_id": "2"}},
)

AIMessage(content='Hello again! I\'m glad you\'re here, Rishi. I\'m here to help you with any questions or concerns you might have. Is there anything specific you would like to know about our medical platform or any other topic? 😊 \n\n(Note: I assumed you might have meant "Hi" as a greeting, so I responded accordingly. If you meant something else, please let me know!) ')